# Assignment 1: Auto-Correct

Welcome to the first assignment of Course 2. This assignment is here to help you brush up on your python and probability skills. In doing so, you will implement an auto-correct system that is very effective and useful. By the end, you will be surprised by the accuracy of your model. 
You use autocorrect everyday in your cell phone and computer. In this assignment, you will explore what really goes on behind the scenes. Of course, the model you are about to implement is not identical to the one used in your phone, but it is good enough to surprise you. By completing this assignment you will learn how to: 

- Get a word count given a corpus
- Get a word probability in the corpus 
- Manipulate strings 
- Filter strings 
- Implement Minimum edit distance to compare strings and to help find the optimal path for the edits. 
- Understand how dynamic programming works


Similar systems are used everywhere. For example, if you type in the word **"I am lerningg"**, chances are very high that you meant to write **"learning"**, as shown in **Figure 1**. In this assignment, you will learn how to come up with that specific probability and in the next assignment you will learn how predict the word **"from"** given that **"learning"** was already written. This will help lay the foundations for more sophisticated NLP tasks, and would allow you to understand natural language processing better when implementing systems like machine translation and speech recognition in Course 4 and 5.




<div style="width:image width px; font-size:100%; text-align:center;"><img src='auto-correct.png' alt="alternate text" width="width" height="height" style="width:300px;height:250px;" /> Figure 1 </div>

In this assignment, you will implement models that correct words that are 1 and 2 edit distances away. 
We say two words are n edit distance away from each other when we need n edits to change one word into another. An edit could consist of one of the following options: 

    - add a letter: ‘te’ => ‘the, ten, ate, ...’
    - remove a letter: ‘hat’ => ‘at, ha, ht’
    - substitute a letter: ‘jat’ => ‘hat, rat, cat, mat, ...’
    - switch two adjacent letters: ‘eta’ => ‘eat, tea,...’

You will be using the four methods above to implement an Auto-correct. To do so, you will need to compute probabilities of a certain word being correct given an input. This auto-correct you are about to implement was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. The goal of the model is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{1}$$

The equation above is essentially just bayes rule. Equation 1 says that the probability of a word being correct is equal to the probability of having a certain word given that it is correct time the probability of being correct divided by the probability of a word appearing. To compute equation one we will first import a data set and then create all the probabilities we need using that data set. 

# Part 1: Data Preprocessing 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

As in any other machine learning task, the first thing you have to do is process your data set. Many courses load in the data for you. However, in the real world you will be all alone creating these NLP systems. Therefore, we feel like it is important for you to load in the dataset and process it alone. Your first task is to read in a file called 'shakespeare.txt' which is found on your Coursera hub. To look at this file you can go to File ==> Open. 

**Instructions**: Implement the function `process_data` which reads in a corpus, changes everything to lower case, and returns a list of words. 

In [2]:
# GRADED FUNCTION: process_data
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in a corpus in lower case. 
    """
    words = [] # return this variable correctly

    ### START CODE HERE ### 
    
    data = open(file_name).read()
    words = re.findall(r'\w+', data.lower())

    ### END CODE HERE ###
    
    return words

In [3]:
words = process_data('shakespeare.txt')
vocab = set(words)  # this will be your new vocabulary and remove the duplicate
# print (words.index('nature'))
# indexes = [i for i,j in enumerate(words) if j == 'nature']
# print (indexes)
print(words[2567])

nature


***Expected Output:*** nature

You now have a list of words that you just imported. You will start by implementing a get_count function that returns a dictionary mapping from the word to its counts in the corpus. For example, given the following sentence: **"I am happy because I am learning"**, your dictionary should return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>


**Instructions**: 
Implement a `get_count` which returns the a dictionary where the key is a word and the value is its frequency in a list of words.  

In [4]:
# GRADED FUNCTION: word_count
def get_count(words):
    '''
    Input:
        words: a list of words representing the corpus. 
    Output:
        word_count_dict: The key is the word and value is the frequency.
    '''
    
    word_count_dict = {}  # return this variable correctly
    ### START CODE HERE 
    word_count_dict = Counter(words)
    ### END CODE HERE ### 
    return word_count_dict

In [5]:
word_count_dict = get_count(words)
print(len(word_count_dict))

6116


In [6]:
# print (word_count_dict['muse'])
# print (word_count_dict.most_common(50))

**Expected Output:**
6116

The next function you will implement will take in the dictionary and compute the probability of each word happening. To compute the probability of a certain word happening, you have to calculate:
$$P(w_i) = \frac{C(w_i)}{|V|} \tag{2}$$
Where $|V|$ is the cardinality, or unique number of words in your corpus. $C(w_i)$ is the total number of times $w_i$ appears in the corpus. For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'**:

$$P(am) = \frac{C(w_i)}{|V|} = \frac {2}{7} \tag{3}.$$

**Instructions:** Implement `get_probs` function which gives you the probability 
of a word happening. This returns a dictionary mapping from the word to its probability in the corpus.

In [7]:
# GRADED FUNCTION: get_probs
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: a dictionary where the keys are the words and their frequencies are the values
    Output:
        probs: a dictionary where the keys are the words and the values are the probability of each word occuring. 
    '''
    probs = {}  # return this variable correctly
    
    ### START CODE HERE ###
    
    n = sum(word_count_dict.values())
    for w in word_count_dict.keys():
        probs[w] = word_count_dict[w] / n

    ### END CODE HERE ###
    return probs

In [8]:
probs = get_probs(word_count_dict)
print(probs['the'])

0.028444063117842356


**Expected Output**: 0.028444063117842356. You can see that the word 'the' has around a 5% chance of happening which is very logical. 

# Part 2: String Manipulations

Now, that you have computed $P(w_i)$ for all the words in the corpus, You will write a few functions to manipulate strings so that you can edit the errors and return the right spelling mistake. In this section, you will implement four functions: 

* deletes: given a word, it returns all the possible words that are in the vocabulary that have one missing character. 
* switches: given a word, it returns all the possible words that are in the vocabulary that have two adjacent letters  switched.
* inserts: given a word, it returns all the possible words that are in the vocabulary that have an additional character inserted in some index.
* replaces: given a word, it returns all the possible words that are in the vocabulary that have one character being replaced in some index.


**Instructions:** Implement a `deletes` function that given a certain word, returns a set of strings with one possible delete. For example, given the word 'nice' it would return the set: {'ice', 'nce', 'nic', 'nie'}. Make sure you only return the words that are in the vocabulary.

In [10]:
# This is a sample test
word_test = 'nice'
L_test = word_test[:4]    #index 0-1
R_test = word_test[4:]    #index 2:length(word_test - 1)
print (L_test)
print (R_test)
print (R_test[1:])
print (L_test + R_test[1:])

nice


nice


In [13]:
# GRADED FUNCTION: deletes
def deletes(word, vocab):
    '''
    Input:
        word: string
    Output:
        deletes: a set of all possible, make sure the possible words in vocabulary 
    '''
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]    # This is to remove index i, NOTE: 'If' condition can also be len(R) > 1 See above exmpale.
    deletes = set(deletes).intersection(set(vocab))    # Frank comment: vocab is already a set, no need to set it again. 
    return set(deletes)    # Frank comment: make it as set again???

In [12]:
# testing your implementation
deletes('nice', vocab)

{'ice'}

**Instructions**: Now implement a function that switches two letters in a word. It takes in a word and returns all the possible switches that are adjacent. For example, given the word eta, it returns {'eat', 'tea'}. Make sure you only return the words that are in the vocabulary. 

In [14]:
# GRADED FUNCTION: switches
def switches(word, vocab):
    '''
    Input:
        word: string
    Output:
        switches: a set of all possible words with one switches 
    ''' 
    switches = set()   # set this correctly
    ### START CODE HERE ###
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    switches = [L + R[1] + R[0] + R[2:] for L, R in splits if len (R)>1]
    switches = set(switches).intersection(set(vocab))
    ### END CODE HERE ###
    return switches

In [15]:
# testing your implementation
switches('eta', vocab)

{'eat'}

**Instructions**: Now you should implement a function that takes in a word and returns a set of words with one replaced letter from the original word. Make sure that you only return the words that are in the vocabulary.

In [16]:
# GRADED FUNCTION: replaces
def replaces(word, vocab):
    '''
    Input:
        word: string
    Output:
        replaces: a set of all possible words where we replaced one letter from the original word. 
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replaces= set() # set this correctly
    ### START CODE HERE ###
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    replaces = set(replaces).intersection(set(vocab))


    ### END CODE HERE ###
    return set(replaces)

In [13]:
replaces('nice',vocab)

{'nice', 'nile', 'nine', 'vice'}

**Expected Output**: {'nice', 'nile', 'nine', 'vice'}

**Instructions**: Now you will implement a function that takes in a word and inserts a letter at every index. It then returns the new set. 

In [17]:
# GRADED FUNCTION: inserts
def inserts(word, vocab):
    '''
    Input:
        word: string
    Output:
        inserts: a set of all possible words with one new letter inserted at every index
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    ### START CODE HERE ###
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    inserts = [L + c + R for L, R in splits for c in letters]
    inserts = set(inserts).intersection(set(vocab))


    ### END CODE HERE ###
    return set(inserts)

In [18]:
inserts('at', vocab)

{'act', 'alt', 'art', 'cat', 'eat', 'sat'}

# Part 3: Combining the edits

Now that you have implemented the string manipulations, you will create two functions that given a word would return all the possible single edits on that word. You will then use that function to generate two edits. You can keep using it for as many edits as you want, but for this assignment you will only perform two edits on an error. 

**Instructions**: Implement the `one_edit` function to get all the possible edits that are one edit away from a word. The edits could consist of either a replace, insert, delete, or switch operation. You should use the previous functions you have already implemented to complete this function. 

In [19]:
# GRADED FUNCTION: one_edit
def one_edit(word, vocab):
    """
    Input:
        word: a string
        vocab: a set of vocabulary in your corpus
    Output:
        one_edits: a set of words with one possible edit, all the words are in the vocabulary.
    """
    ### START CODE HERE ###
    one_edits = inserts(word, vocab) | replaces(word,vocab) | switches(word, vocab) | deletes(word, vocab)
#     one_edits = inserts(words, vocab).union(replaces(word,vocab))
    ### END CODE HERE ###
    
    return one_edits

In [28]:
s = one_edit("yours", vocab)
print(s)

{'ours', 'hours', 'yours', 'your'}


Now you can generalize this to implement to get two edits on a word. To do so, you would have to get all the possibble edits on a single word and then for each modified word, you would have to modify it again. 

**Instructions**: Implement the two_edits function that returns a list of all words that are two edits away. 

In [36]:
# GRADED FUNCTION: two_edits
def two_edits(word, vocab):
    '''
    Input:
        word: a string
        vocab: a set of vocabulary in your corpus
    Output:
        one_edits: a set of words with one possible edit, all the words are in the vocabulary.
    '''
    final_words = set()
    edits = one_edit(word,vocab)
    final_words = final_words | edits
    for cur_word in edits:
        new_set = one_edit(cur_word,vocab)
        final_words = final_words | new_set    # update the final words set
    return final_words

In [44]:
# Test if your implementation makes sense
two_edits("mother", vocab)

{'mother', 'mothers', 'other', 'others', 'smother'}

**Expected Output**: {'sneaking', 'speaking', 'stealing'}

Now you will use your `two_edits` function to get a set of all the possible 2 edits on your word. You will then use those words to get the most probable word you meant to type.
**Instructions**: Implement `get_corrections`, which returns the word if it is found in the vocabulary and a dictionary of n possible suggestions (where the key is the proposed word and the value is its probability) otherwise. You should use your two_edits functions. You can also use `most_common` which is a helper function inside Counter. Good luck. 

In [48]:
# dict_test = {"abc":0.1, "bcd": 0.4, "cde":0.2}
# dict_count = Counter(dict_test)
# dict_count.most_common(1)

In [49]:
# GRADED FUNCTION: get_corrections
def get_corrections(word, probs, vocab, n=2):
    '''
    Input: 
        word: a string 
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        best_words: a dictionary with the most probable n corrected words. The values are the proabilities.
    '''
    
    ### START CODE HERE ###
    if word in vocab:
        return word
    s = two_edits(word, vocab)
    best_words = {}
    for w in s:
        curr_probability = probs[w]
        best_words[w] = curr_probability
    best_words = Counter(best_words)    # key: word; value: frequency
    best_words = best_words.most_common(n)
    ### END CODE HERE ###

    return best_words

In [51]:
# Test your implementation - feel free to try other words in my word
my_word = 'mahs' 
get_corrections(my_word,probs, vocab, 2)

[('wars', 0.00031708135934643936), ('mars', 0.00022382213600925132)]

# Part 4: Minimum Edit distance

Now that you have implemented your auto-correct, How do you evaluate the similarity between two strings for example: 'waht' and 'what'? Also how do you efficiently find the shortest path to go from the word, 'waht' to the word 'what'? You will implement a dynamic programming system that will not only tell you the minimum amount of edits required to convert a string into another string, but also tells you in what order to convert these strings. In lecture, you saw minimum edit distance. You have to create a matrix and update each element in the matrix as follows:

$$ D[i,j]=
\begin{cases}
D[i-1,j] + del-cost(source[i])\\
D[i-1,j-1] + sub\_cost(source[i],target[j])\\
D[i,j-1] + ins-cost(target[j])
\end{cases} \tag{4}$$
So converting the word **play** to **stay**, using an input cost of one, a delete cost of 1, and replace cost of 2 would give you the following table:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>



**Instructions**: Implement the function below to get the minimum amount of edits required given a source string and a target string. 

In [52]:
# GRADED FUNCTION: min_edit_distance
def min_edit_distance(source, target):
    '''
    Input: 
        source: a string corresponding to the word you are starting with
        target: a string corresponding to the word you want to end with
    Output:
        D: a matrix of len(source) by len(target)
        D[m,n]: the minimum number of edits required to convert the source string to the target
        
    '''
    # use del_cost = 1, ins_cost = 1, and rep_cost = 2
    del_cost = 1
    ins_cost = 1
    m = len(source)
    n = len(target)
    D = np.zeros((m+1, n+1), dtype=int) # row 0 and column 0 is the distance from the empty string
    ### START CODE HERE ###
    for row in range(1,m+1):
        D[row,0] = D[row-1,0] + del_cost
    for col in range(1,n+1):
        D[0,col] = D[0,col-1] + ins_cost
    for row in range(1,m+1):
        for col in range(1,n+1):
            rep_cost = 2
            if source[row-1] == target[col-1]:
                rep_cost = 0
            # delete, insert, replace
            D[row,col] = min(D[row-1,col] + 1, D[row,col-1]+1, D[row-1,col-1] + rep_cost) # get the matrix
    ### END CODE HERE ###
    return D, D[m, n]

In [53]:
# testing your implementation
source =  'plays'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4
s  5  4  5  6  5


# Part 5: Optional - Backtrace

Once you have computed your matrix using minimum edit distance, how would find the shortest path from the top left corner to the bottom right corner? Note you could use backtrace. Make sure you submit your assignment before you modify anything. Try to find the shortest path given the matrix that your `min_edit_distance` function returned.

In [24]:
# Experiment with back trace - insert your code here



###### References
- Dan Jurafsky - Speech and Language Processing - Textbook
- This auto-correct explanation was first done by Peter Norvig in 2007 